In [ ]:
from datetime import datetime
from dataretrieval import nwis
from metloom.pointdata import SnotelPointData
from metloom.variables import SnotelVariables

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

startDate = '2000-10-01'
endDate = '2025-09-30'

startDate_dt = datetime.strptime(startDate, '%Y-%m-%d')
endDate_dt = datetime.strptime(endDate, '%Y-%m-%d')

boise_river_outp = 'boise_river_snow2flow.csv'

# NWIS Variables
gage = '13185000' # Boise River Near Twin Springs

# Metloom Variables
snotel_stn = '830:ID:SNTL' # Trinity Mountain Snotel Station
snotel_pnt = SnotelPointData(station_id=snotel_stn, name='TrinityMtn')

variables = [snotel_pnt.ALLOWED_VARIABLES.SWE,
             snotel_pnt.ALLOWED_VARIABLES.TEMPAVG, 
             snotel_pnt.ALLOWED_VARIABLES.PRECIPITATION]

In [ ]:
df_snotel = snotel_pnt.get_daily_data(startDate_dt, endDate_dt, variables)
df_snotel

In [ ]:
df_streamflow = nwis.get_dv(sites=gage, parameterCd='00060', start=startDate, end=endDate)[0]
df_streamflow


In [ ]:
df_snotel['SWE'].plot()
df_streamflow['00060_Mean'].plot()

1. Create a date range for the combined dataframe
2. Get the data from the snotel dataframe
3. Convert units from snotel data
4. Get the data from the streamflow dataframe
5. Convert units for streamflow data
6. Export combined dataframe to a csv file (for later)

In [ ]:
combined_dt = pd.date_range(start=startDate_dt, end=endDate_dt, freq='1D')
combined_dt

In [ ]:
combined_swe = df_snotel['SWE'].values
combined_swe = combined_swe * 25.4 # in to mm conversion

combined_tavg = df_snotel['AVG AIR TEMP'].values
combined_tavg = (combined_swe - 32.0)*(5/9) # °F to °C conversion

combined_prec = df_snotel['PRECIPITATION'].values
combined_prec = combined_prec * 25.4 # in to mm conversion

In [ ]:
combined_q = df_streamflow['00060_Mean'].values
combined_q = combined_q * (1/(3.2808**3))

In [ ]:
combined_data = {
    'datetime': combined_dt,
    'prec_mm': combined_prec,
    'tavg_degc': combined_tavg,
    'swe_mm': combined_swe,
    'flow_cms': combined_q,
}


In [ ]:
df_combined = pd.DataFrame(combined_data)
df_combined = df_combined.set_index('datetime')
df_combined.to_csv(boise_river_outp)
df_combined

In [ ]:
df_combined.plot()